## Knowledge Bases - Reranker with metadata

---

Reranker can soft boost on metadata field. Checkout [AWS user guide](https://docs.aws.amazon.com/bedrock/latest/userguide/kb-metadata.html) to understand how to ingest metadata into Knowledgebase.

This notebook will provide a simple example on how to use reranker to rerank on a specific filed of metadata. 

#### Pre-requisite: 

1. Go through `01-reranker-evaluate-ragas.ipynb` to understand basic reranker setup
2. Create a new knowledge base and ingest data included in the `sec-10k` folder
3. Note the Knowledge Base ID

---


In [1]:
kb_id = "YWU2HLLTET" # Replace with your knowledge base id here.

# kb_id = "<<knowledge_base_id>>" # Replace with your knowledge base id here.

In [2]:
import boto3
import pprint
from botocore.client import Config
import time
import json


bedrock_client = boto3.client('bedrock-runtime')

# TODO remove the gamma endpoint when publishing
bedrock_agent_client = boto3.client(
    "bedrock-agent-runtime",
    endpoint_url="https://gamma.us-east-1.runtime.bedrock-agent.aws.dev/",
    region_name="us-east-1",
)

TEXT_GENERATION_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
EVALUATION_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"


# Reranker model: there are two reranker models available at launch
AMAZON_RERANKER_MODEL = "arn:aws:bedrock:us-east-1::foundation-model/amazon.rerank-v1:0"
COHERE_RERANKER_MODEL = "arn:aws:bedrock:us-east-1::foundation-model/cohere.rerank-v3-5:0"


In [3]:
def retrieve_docs(query, retrieval_config):
    start = time.time()
    response = bedrock_agent_client.retrieve(
        retrievalQuery={
            'text': query
        },
        knowledgeBaseId=kb_id,
        retrievalConfiguration=retrieval_config
    )
    time_spent = time.time() - start
    print(f"[Retrieved docs]\n{json.dumps(response['retrievalResults'], indent=2)}\n")
    print(f"[Invocation time]\n{time_spent}\n")

    return response

---

### Retrieve without metadata

---

In [4]:
query = "what's amazon's revenue in 2020?"

In [5]:
retrieval_config_without_metadata = {
    "vectorSearchConfiguration": {
        "numberOfResults": 100,
        "rerankingConfiguration": {
            "type": "BEDROCK_RERANKING_MODEL",
            "bedrockRerankingConfiguration": {
                "modelConfiguration": {
                    "modelArn": AMAZON_RERANKER_MODEL
                },
                "numberOfRerankedResults": 3
                }
        }
    }
}

In [6]:
retrieve_docs(query, retrieval_config_without_metadata)

[Retrieved docs]
[
  {
    "content": {
      "text": "Conversely, our Consumer revenue grew dramatically in 2020. In 2020, Amazon\u2019s North America and International Consumer revenue grew 39% YoY on the very large 2019 revenue base of $245 billion; and, this extraordinary growth extended into 2021 with revenue increasing 43% YoY in Q1 2021. These are astounding numbers. We realized the equivalent of three years\u2019 forecasted growth in about 15 months.     As the world opened up again starting in late Q2 2021, and more people ventured out to eat, shop, and travel, consumer spending returned to being spread over many more entities. We weren\u2019t sure what to expect in 2021, but the fact that we continued to grow at double digit rates (with a two-year Consumer compounded annual growth rate of 29%) was encouraging as customers appreciated the role Amazon played for them during the pandemic, and started using Amazon for a larger amount of their household purchases.     This growth 

{'ResponseMetadata': {'RequestId': '4164ba4f-dea4-42fb-99c5-2eb714cf5184',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 27 Nov 2024 14:45:12 GMT',
   'content-type': 'application/json',
   'content-length': '4792',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4164ba4f-dea4-42fb-99c5-2eb714cf5184'},
  'RetryAttempts': 0},
 'retrievalResults': [{'content': {'text': 'Conversely, our Consumer revenue grew dramatically in 2020. In 2020, Amazon’s North America and International Consumer revenue grew 39% YoY on the very large 2019 revenue base of $245 billion; and, this extraordinary growth extended into 2021 with revenue increasing 43% YoY in Q1 2021. These are astounding numbers. We realized the equivalent of three years’ forecasted growth in about 15 months.     As the world opened up again starting in late Q2 2021, and more people ventured out to eat, shop, and travel, consumer spending returned to being spread over many more entities. We weren’t sure what to expect 

---

### Retrieve with metadata

**Note:** please observe how the retrieved chunks get changed when applying metadata in the reranker configuration especially the metadata `year` included in the retrieved chunks.

---

In [7]:
retrieval_config_with_metadata = {
    "vectorSearchConfiguration": {
        "numberOfResults": 100,
        "rerankingConfiguration": {
            "type": "BEDROCK_RERANKING_MODEL",
            "bedrockRerankingConfiguration": {
                "modelConfiguration": {
                    "modelArn": AMAZON_RERANKER_MODEL
                },
                "numberOfRerankedResults": 3,
                "metadataConfiguration" : {
                    "selectionMode" : "SELECTIVE",
                    "selectiveModeConfiguration" : {
                        "fieldsToInclude": [{
                            "fieldName": "year",
                        }]
                    }
                }
            }
        }
    }
}

In [8]:
retrieve_docs(query, retrieval_config_with_metadata)

[Retrieved docs]
[
  {
    "content": {
      "text": "Conversely, our Consumer revenue grew dramatically in 2020. In 2020, Amazon\u2019s North America and International Consumer revenue grew 39% YoY on the very large 2019 revenue base of $245 billion; and, this extraordinary growth extended into 2021 with revenue increasing 43% YoY in Q1 2021. These are astounding numbers. We realized the equivalent of three years\u2019 forecasted growth in about 15 months.     As the world opened up again starting in late Q2 2021, and more people ventured out to eat, shop, and travel, consumer spending returned to being spread over many more entities. We weren\u2019t sure what to expect in 2021, but the fact that we continued to grow at double digit rates (with a two-year Consumer compounded annual growth rate of 29%) was encouraging as customers appreciated the role Amazon played for them during the pandemic, and started using Amazon for a larger amount of their household purchases.     This growth 

{'ResponseMetadata': {'RequestId': '9314e209-7259-4552-a636-339c785d1e01',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 27 Nov 2024 14:45:14 GMT',
   'content-type': 'application/json',
   'content-length': '5246',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9314e209-7259-4552-a636-339c785d1e01'},
  'RetryAttempts': 0},
 'retrievalResults': [{'content': {'text': 'Conversely, our Consumer revenue grew dramatically in 2020. In 2020, Amazon’s North America and International Consumer revenue grew 39% YoY on the very large 2019 revenue base of $245 billion; and, this extraordinary growth extended into 2021 with revenue increasing 43% YoY in Q1 2021. These are astounding numbers. We realized the equivalent of three years’ forecasted growth in about 15 months.     As the world opened up again starting in late Q2 2021, and more people ventured out to eat, shop, and travel, consumer spending returned to being spread over many more entities. We weren’t sure what to expect 